In [1]:
!pip install sqlalchemy beautifulsoup4 selenium requests


  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 2.4 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.3 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.5 MB 2.4 MB/s et


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries

In [23]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from sqlalchemy import create_engine
from selenium.webdriver.chrome.options import Options



In [51]:
url = 'https://books.toscrape.com/'

r = requests.get(url)

print(r)

<Response [200]>


In [75]:
# configure selenium driver
options = Options()

service = Service(executable_path=r"C:\Users\OLALEKAN\Downloads\chromedriver-win64\chromedriver.exe" )

#initialize webdriver
driver = webdriver.Chrome(service=service, options=options)

# Define the url

url = "https://books.toscrape.com/catalogue/category/books_1/index.html"

driver.get(url)


page_source = driver.page_source

time.sleep(20)

driver.quit()


In [53]:
soup = BeautifulSoup(page_source, 'html.parser')


In [54]:
soup

<html class="no-js" lang="en-us"><!--<![endif]--><head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="static/oscar/js/bootstrap-datetimepicker/bootstrap-datetimepicker.css" rel="stylesheet"/>
<link href="static/oscar/css/datetimepicker.css" rel="stylesheet" type="text/css"/>
</head>
<body class="default" id="default">
<header class="header container-fluid">
<div class="page_inner">
<div class="

In [55]:
ol = soup.find('ol')

In [59]:
articles = ol.find_all('article', class_='product_pod')

In [81]:
books = []

# Loop through multiple pages
for i in range(1, 5):
    url = f"https://books.toscrape.com/catalogue/page-{i}.html"  # Correct URL for pagination
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve page {i}")
        continue  # Skip to next iteration if the page fails to load

    soup = BeautifulSoup(response.content, 'html.parser')
    ol = soup.find('ol')

    if not ol:  # Ensure 'ol' is found before proceeding
        print(f"No books found on page {i}")
        continue

    articles = ol.find_all('article', class_='product_pod')  # Corrected class syntax

    for article in articles:
        image = article.find('img')
        title = image.attrs['alt'] if image else "No title"

        review = article.find('p')
        review = review['class'][1] if review and 'class' in review.attrs else "No review"

        price = article.find('p', class_='price_color')
        price = float(price.text[1:]) if price else 0.0

        books.append([title, price, review])

In [82]:
df = pd.DataFrame(books, columns=['Title', 'Price', 'Review'])

In [83]:
df.to_csv('new_books.csv')